# target densities for different BGS TS scenarios

In [1]:
import numpy as np 
from astropy.io import fits
import matplotlib.pyplot as plt

In [2]:
from desitarget.sv1.sv1_targetmask import desi_mask as sv1_desi_mask

In [3]:
targets = fits.open('/global/cfs/cdirs/desi/target/catalogs/dr9/0.49.0/targets/sv1/resolve/bright/sv1targets-bright-hp-399.fits')[1].data

In [4]:
radec_lim = (targets['RA'] > 178) & (targets['RA'] < 182.) & (targets['DEC'] > -6) & (targets['DEC'] < -2)
targets = targets[radec_lim]

In [5]:
randoms = fits.open('/global/cfs/cdirs/desi/target/catalogs/dr9/0.47.0/randoms/resolve/randoms-allsky-0.fits')[1].data

In [6]:
radec_lim = (randoms['RA'] > 178) & (randoms['RA'] < 182.) & (randoms['DEC'] > -6) & (randoms['DEC'] < -2)
randoms = randoms[radec_lim]

In [7]:
# get effective area from randoms
def spatial_mask(maskbits, nobs):
    ''' spatial masking around
    
    * bright stars
    * medium bright stars
    * clusters
    * large galaxies
    '''
    nobs_g, nobs_r, nobs_z = nobs
    BS = (np.uint64(maskbits) & np.uint64(2**1))!=0    # bright stars
    MS = (np.uint64(maskbits) & np.uint64(2**11))!=0   # medium bright stars
    GC = (np.uint64(maskbits) & np.uint64(2**13))!=0   # clusters
    LG = (np.uint64(maskbits) & np.uint64(2**12))!=0   # large galaxies
    allmask = ((maskbits & 2**6) != 0) | ((maskbits & 2**5) != 0) | ((maskbits & 2**7) != 0)
    nobs = ((nobs_g < 1) | (nobs_r < 1) | (nobs_z < 1))
    mask = ~(BS | MS | GC | LG | allmask | nobs)
    return mask

In [8]:
_spatial = spatial_mask(randoms['MASKBITS'], [randoms['NOBS_G'], randoms['NOBS_R'], randoms['NOBS_Z']])
area = 16.
eff_area = float(np.sum(_spatial)) / float(randoms.shape[0]) * area

In [9]:
print('4degx4degree patch with effective area %.1f' % eff_area)

4degx4degree patch with effective area 15.4


In [10]:
isbgs = ((targets['SV1_DESI_TARGET'] & sv1_desi_mask['BGS_ANY']) != 0)
bgs = targets[isbgs]

Now we can calculate target densities

In [11]:
# compile some properties
g_mag = (22.5 - 2.5*np.log10(bgs['FLUX_G']))
r_mag = (22.5 - 2.5*np.log10(bgs['FLUX_R']))
z_mag = (22.5 - 2.5*np.log10(bgs['FLUX_Z']))
w1_mag = (22.5 - 2.5*np.log10(bgs['FLUX_W1']))
r_fibmag = (22.5 - 2.5 * np.log10(bgs['FIBERFLUX_R']))

schlegel_color = (z_mag - w1_mag) - 3/2.5 * (g_mag - r_mag) + 1.2

<ipython-input-11-d5f8fd16be7a>:2: RuntimeWarning: divide by zero encountered in log10
  g_mag = (22.5 - 2.5*np.log10(bgs['FLUX_G']))
<ipython-input-11-d5f8fd16be7a>:2: RuntimeWarning: invalid value encountered in log10
  g_mag = (22.5 - 2.5*np.log10(bgs['FLUX_G']))
<ipython-input-11-d5f8fd16be7a>:4: RuntimeWarning: divide by zero encountered in log10
  z_mag = (22.5 - 2.5*np.log10(bgs['FLUX_Z']))
<ipython-input-11-d5f8fd16be7a>:4: RuntimeWarning: invalid value encountered in log10
  z_mag = (22.5 - 2.5*np.log10(bgs['FLUX_Z']))
<ipython-input-11-d5f8fd16be7a>:5: RuntimeWarning: invalid value encountered in log10
  w1_mag = (22.5 - 2.5*np.log10(bgs['FLUX_W1']))


In [12]:
rbright_lim = (r_mag < 19.5)
rfaint_lim = (r_mag > 19.5) & (r_mag < 20.) 
rfainter_lim = (r_mag > 20.) & (r_mag < 20.2) 
rfainterer_lim = (r_mag > 20.2) & (r_mag < 20.4) 
rfaintest_lim = (r_mag > 20.4) 
rfib_lim = (r_fibmag < 22)
fibcol_cut = (r_fibmag < 20.5) | ((r_fibmag < 21.5) & (schlegel_color > 0.))

# BGS nominal target densities

In [13]:
# r < 19.5 
print(np.sum(rbright_lim) / eff_area)
# 19.5 < r < 20.
print(np.sum(rfaint_lim) / eff_area)
# 20.0 < r < 20.2
print(np.sum(rfainter_lim) / eff_area)
# 20.2 < r < 20.4
print(np.sum(rfainterer_lim) / eff_area)
# 20.4 < r
print(np.sum(rfaintest_lim) / eff_area)

894.7476454868588
627.1941840930908
357.7560857354124
414.10023220818306
385.9606527125548


# BGS $r_{\rm fib} < 22$ target densities

In [14]:
# r < 19.5 
print(np.sum(rfib_lim & rbright_lim) / eff_area)
# 19.5 < r < 20.
print(np.sum(rfib_lim & rfaint_lim) / eff_area)
# 20.0 < r < 20.2
print(np.sum(rfib_lim & rfainter_lim) / eff_area)
# 20.2 < r < 20.4
print(np.sum(rfib_lim & rfainterer_lim) / eff_area)
# 20.4 < r
print(np.sum(rfib_lim & rfaintest_lim) / eff_area)

883.9597235555094
605.8782901564487
336.76512920634093
381.2165665620216
334.4255798718315


# BGS fiber-color cut target densities

In [15]:
# r < 19.5 
print(np.sum(fibcol_cut & rbright_lim) / eff_area)
# 19.5 < r < 20.
print(np.sum(fibcol_cut & rfaint_lim) / eff_area)
# 20.0 < r < 20.2
print(np.sum(fibcol_cut & rfainter_lim) / eff_area)
# 20.2 < r < 20.4
print(np.sum(fibcol_cut & rfainterer_lim) / eff_area)
# 20.4 < r
print(np.sum(fibcol_cut & rfaintest_lim) / eff_area)

762.4331331240433
337.4150040214825
146.091858443816
137.96842325454688
98.13109708637106
